## Precarga de librerias y funciones

In [ ]:
%%capture
pip install shap ;

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: ''


In [ ]:
%%capture
pip install openpyxl ;

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: ''


In [ ]:
%%capture
pip install imblearn ;

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: ''


In [ ]:
import pandas as pd
import numpy as np

#visualizacion

import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#modelos
import lightgbm as lgb
from lightgbm.callback import early_stopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import f1_score
from sklearn.ensemble import  RandomForestClassifier
from sklearn.impute import SimpleImputer
from joblib import Parallel, delayed
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import fbeta_score, make_scorer
from imblearn.under_sampling import TomekLinks



#base de datos
import duckdb
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

#optimizacion
import time
import optuna
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState
from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history
import plotly.express as px

#kaggle y otros
import os
import kaggle 
import glob
from openpyxl import load_workbook
from datetime import datetime
import contextlib



#Shap values
import shap

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
import random

numeros_random = random.sample(range(1, 1000), 35)
semillas =[ 400009,  500009, 500011, 500021, 600009]


## Paths

### VM

In [ ]:
base_path = '/home/n_alba_dscience/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
datasets_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
save_path = base_path + 'buckets/b1/'


# 1 - dataset sólo con clase ternaria
dataset_clase_ternaria = datasets_path + 'competencia_03_ct.parquet'
dataset_clase_ternaria_sin_PCI = datasets_path + 'competencia_03_ct_sin_PCI.parquet'
dataset_competencia_03_lags_deltas_y_clase_ternaria = datasets_path + 'competencia_03_lags_deltas_y_clase_ternaria.parquet'


### Local

In [ ]:
base_path_l = r'C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF'
optuna_path_l = base_path_l + r'\Optuna\rf_segmentacion1\optimization_tree.db'

entregas_l = base_path_l + r'\Entregas\competencia_03'


db_path_l = base_path_l + r'datasets\competencia_3'

datasets_path_l = base_path_l + r'\datasets\competencia_3'



# Competencia 0
# 1 dataset sólo con clase ternaria
dataset_clase_ternaria_l = datasets_path_l + r'\competencia_03_ct.parquet'
dataset_clase_ternaria_sin_psi_l = datasets_path_l + r'\competencia_03_ct_sin_psi.parquet'
dataset_competencia_03_lags_deltas_y_clase_ternaria_l = datasets_path_l + r'\competencia_03_lags_deltas_y_clase_ternaria.parquet'
# 2 dataset con clase ternaria creada y feature engineering de lags 




## Funciones

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [ ]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    breakpoints = sorted(set(
    [edge.left for edge in bin_edges if isinstance(edge, pd.Interval)] + 
    [edge.right for edge in bin_edges if isinstance(edge, pd.Interval)]))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [ ]:
# Creador clase ternaria

# df_parallel = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01_crudo.csv")

# start_time = time.time()

# # Crear la columna clase_ternaria usando la columna 'foto_mes' basada en la presencia real

# df_ternaria= duckdb.query('''

#     WITH siguiente_mes AS (

#         SELECT

#             numero_de_cliente,

#             foto_mes,

#             -- Obtener los meses siguientes en los que el cliente está presente

#             LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo1,

#             LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo2

#         FROM df_parallel

#     )

#     SELECT

#         numero_de_cliente,

#         foto_mes,

#         CASE

#             -- Si el cliente está en los dos meses consecutivos siguientes

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND foto_mes_proximo2 = foto_mes + 2 THEN 'CONTINUA'

#             -- Si el cliente no está en el siguiente mes

#             WHEN foto_mes_proximo1 IS NULL OR foto_mes_proximo1 <> foto_mes + 1 THEN 'BAJA+1'

#             -- Si el cliente está en el siguiente mes pero no en el segundo mes consecutivo

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND (foto_mes_proximo2 IS NULL OR foto_mes_proximo2 <> foto_mes + 2) THEN 'BAJA+2'

#             ELSE NULL

#         END AS clase_ternaria

#     FROM siguiente_mes

#     ORDER BY numero_de_cliente, foto_mes

#     ''')

# df_ternaria = df_ternaria.to_df()
# df_ternaria = df_ternaria[['numero_de_cliente', 'foto_mes', 'clase_ternaria']]


# df_parallel = df_parallel.merge(df_ternaria, on=['numero_de_cliente', 'foto_mes'], how='left')

# df_train = df_parallel


In [ ]:

# con = duckdb.connect(database=':memory:', read_only=False)

# ruta_csv = r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\datasets\competencia_2\competencia_02_f_e_lags.parquet"

# con.execute(f"CREATE TABLE df_train AS SELECT * FROM read_parquet('{ruta_csv}')")

In [ ]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    test_data = lgb.Dataset(X_test, label=X_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas

In [ ]:
def anotar_excel(path_excel, dicc_medidas):
    if excel == True:
    
        wb = load_workbook(path_excel)
        ws = wb.active

        header_row = 1  
        column_mapping = {cell.value: cell.column for cell in ws[header_row]}
    
    row_to_update = None

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
        cell = row[0]  # La primera celda de la fila
        if cell.value is None:  # Si el valor de la celda está vacía
            print(f"Primera celda vacía en la fila {cell.row}")
            row_to_update = cell.row
            break

    # Si no se encontró ninguna celda vacía, agregar al final
    if row_to_update is None:
        print("No se encontro celda vacía, agregando al final")
        row_to_update = ws.max_row + 1  # Agregar en una nueva fila al final

    print(f"Fila a actualizar: {row_to_update}")
    fila_actualizar = row_to_update  # Fila donde escribir los datos

    for col_name, value in dicc_medidas.items():
        if col_name in column_mapping:  # Asegurarse de que la columna existe
            col_idx = column_mapping[col_name]
            ws.cell(row=fila_actualizar, column=col_idx, value=value)

    # Guardar los cambios
    wb.save(path_excel)
    
    return print (f"se actualizó fila {fila_actualizar} en el excel")

In [ ]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    test_data = lgb.Dataset(X_test, label=Y_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
     
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas

In [ ]:
def calculoGanancia(bajas,prediccion,corte,seed):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=seed
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado


In [ ]:
def predecir_punto_de_corte (dataset, mes_de_entrenamiento, mes_de_prueba, best_params, best_iteration, rango_cortes, semillas):
    if isinstance(dataset, pd.DataFrame):
        df_train = pd.read_parquet(dataset_path)
    elif isinstance(dataset, str):
        df_train = pd.read_parquet(dataset)
    else: print('dataset debe ser un DataFrame o un string con el path del archivo')
    
    ganancia_acierto = 273000
    costo_estimulo = 7000
    mes_train = mes_de_entrenamiento
    mes_test = mes_de_prueba
    data = df_train
    data['clase_peso'] = 1.0
    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
    data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
    data['foto_mes'].unique()
    if isinstance(mes_de_entrenamiento, list):
        df_train = data[data['foto_mes'].isin(mes_train)]
    else: df_train = data[data['foto_mes']==mes_train]
    df_test = data[data['foto_mes']==mes_test]
    clase_peso = df_train['clase_peso']
    X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
    Y_train =df_train['clase_binaria']
    X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
    Y_test =df_test['clase_binaria']
    w_train = df_train['clase_peso']
    w_test = df_test['clase_peso']
    best_params = best_params
    best_iter = best_iteration    
    train_data = lgb.Dataset(X_train,
                                label=Y_train,
                                weight=w_train)
    best_iter = 1789

    df_voting = pd.DataFrame()

    for x in semillas:
        params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': best_params['num_leaves'],
        'learning_rate': best_params['learning_rate'],
        'min_data_in_leaf': best_params['min_data_in_leaf'],
        'feature_fraction': best_params['feature_fraction'],
        'bagging_fraction': best_params['bagging_fraction'],
        'seed': x,
        'verbose': 0}
            
                
        model = lgb.train(params,
                        train_data,
                        num_boost_round=best_iter)

        y_pred = model.predict(X_test)
        df_voting[f'prediccion_seed_{x}'] = y_pred

    df_voting['prediccion'] = df_voting.mean(axis=1)
    df_voting.index = X_test.index
    bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
    prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
    resultados = []
    cortes = range(5000, 20000, rango_cortes)


    pred_model_sorted = prediccion.sort_values('probabilidad', ascending=False)

    model_name = 'LightGBM'

    # Iteramos sobre cada corte
    for corte in cortes:
        lista_ganancia_publico =[]
        lista_ganancia_privado = []
        for semilla in semillas: 
            ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte, semilla)        
            lista_ganancia_publico.append(ganancia_publico)
            lista_ganancia_privado.append(ganancia_privado)
        promedio_ganancia_publico = np.mean(lista_ganancia_publico)
        promedio_ganancia_privado = np.mean(lista_ganancia_privado)
            
        resultados.append({
        'Modelo': model_name,
        'Corte': corte,
        'Ganancia Público': promedio_ganancia_publico,
        'Ganancia Privado': promedio_ganancia_privado
        })

    # Convertimos los resultados en un DataFrame
    resultados = pd.DataFrame(resultados)
    resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado'])

    # Aplanamos las columnas para facilitar el acceso
    resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

    # Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
    # Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
    columnas_ordenadas = []
    for modelo in resultados['Modelo'].unique():
        columnas_ordenadas.append(f'Ganancia Público_{modelo}')
        columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

    # Reorganizamos el DataFrame usando el nuevo orden de columnas
    resultados_pivot = resultados_pivot[columnas_ordenadas]

    # Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
    resultados_pivot = resultados_pivot.reset_index()
    maxima_ganancia_publico = resultados_pivot['Ganancia Público_LightGBM'].max()
    maxima_ganancia_privado = resultados_pivot['Ganancia Privado_LightGBM'].max()


    for indice, resultado in resultados_pivot.iterrows():
        if resultado['Ganancia Público_LightGBM'] == maxima_ganancia_publico:
            corte_maxima_ganancia_publico = indice  # Puedes ajustar según la lógica de "corte"
            ganancia_si_corte_maxima_ganancia_publico = (
                resultado['Ganancia Público_LightGBM'] + resultado['Ganancia Privado_LightGBM']
            )
            print(f'Ganancia si se hace corte por máxima ganancia público: {ganancia_si_corte_maxima_ganancia_publico}, el corte será {corte_maxima_ganancia_publico}')
        
        if resultado['Ganancia Privado_LightGBM'] == maxima_ganancia_privado:
            corte_maxima_ganancia_privado = indice  # Puedes ajustar según la lógica de "corte"
            ganancia_si_corte_maxima_ganancia_privado = (
                resultado['Ganancia Público_LightGBM'] + resultado['Ganancia Privado_LightGBM']
            )
            print(f'Ganancia si se hace corte por máxima ganancia privado: {ganancia_si_corte_maxima_ganancia_privado}, el corte será {corte_maxima_ganancia_privado}')
                
    corte_publico_ganancia_total = resultados_pivot.loc[corte_maxima_ganancia_publico,'Ganancia Privado_LightGBM'] + resultados_pivot.loc[corte_maxima_ganancia_publico,'Ganancia Público_LightGBM']
    corte_privado_ganancia_total = resultados_pivot.loc[corte_maxima_ganancia_privado,'Ganancia Privado_LightGBM'] + resultados_pivot.loc[corte_maxima_ganancia_privado,'Ganancia Público_LightGBM']
    corte_maxima_ganancia_total = corte_maxima_ganancia_privado if corte_privado_ganancia_total > corte_publico_ganancia_total else corte_maxima_ganancia_publico
    incentivos = resultados_pivot.loc[corte_maxima_ganancia_total,'Corte']
    print (f'El corte recomendado por máxima ganancia total es {corte_maxima_ganancia_total} con un incentivo de {incentivos}')
    return incentivos, resultados_pivot

In [ ]:
def hacer_anotacion_corte (dataset:str, extra:str, df_train:pd.DataFrame, incentivos:int, mensaje:str):
    """
    variables: fecha,hora,dataset,cantidad_variables,corte,extra
    """
    
    fecha = datetime.now().strftime("%Y-%m-%d")
    hora = datetime.now().strftime("%H:%M:%S")
    cantidad_variables = df_train.shape
    
    dataframe = pd.read_csv(r"../../anotaciones_punto_de_corte.csv")
    nueva_fila = pd.DataFrame({'fecha':[fecha],'hora':[hora],'dataset':[dataset],'cantidad_variables':[cantidad_variables],'corte':[incentivos],'extra':[extra],'mensaje':[mensaje]})
    dataframe = pd.concat([dataframe, nueva_fila], ignore_index=True)
    dataframe.to_csv(r"../../anotaciones_punto_de_corte.csv", index=False)
    os.system('git add ../../anotaciones_punto_de_corte.csv')
    os.system(f"git commit -m '{mensaje}'")
    os.system("git push")

In [ ]:
def hacer_anotacion_back_testing (dataset:str, extra:str, df_train:pd.DataFrame, continuas:int, baja_1:int, baja_2:int , mensaje:str):
    """
    variables: fecha,hora,dataset,continuas,baja+1,baja+2,corte,extra
    """
    
    fecha = datetime.now().strftime("%Y-%m-%d")
    hora = datetime.now().strftime("%H:%M:%S")
    cantidad_variables = df_train.shape
    
    dataframe = pd.read_csv(r"../../anotaciones_back_testing.csv")
    nueva_fila = pd.DataFrame({'fecha':[fecha],'hora':[hora],'dataset':[dataset],'cantidad_variables':[cantidad_variables],'continuas':[continuas], 'baja+1':[baja_1],'extra':[baja_2],'mensaje':[mensaje]})
    dataframe = pd.concat([dataframe, nueva_fila], ignore_index=True)
    dataframe.to_csv(r"../../anotaciones_back_testing.csv", index=False)
    os.system('git add ../../anotaciones_back_testing.csv')
    os.system(f"git commit -m '{mensaje}'")
    os.system("git push")

## Parametros modelo

In [ ]:
# 1 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000


mes_train = [201901, 201902, 201903, 201905, 201907, 201911, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107]
mes_test = 202109

best_params=### 

best_iter= ##


## Feature Engineering

In [3]:
# 1 configuración de la base de datos

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
CREATE OR REPLACE TABLE competencia_03 AS
SELECT * FROM read_parquet(:dataset_clase_ternaria_sin_psi_l);

,Success


In [ ]:
%%sql 
SELECT * FROM competencia_03;

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mactivos_margen,mpasivos_margen,cproductos,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,250480925,201901,1,0,39,166,2734.28,-221.90,1424.55,7,...,1.0,0.0,-23237.35,0.0,2177.0,6766.59,9.0,0.0,938.40,CONTINUA
1,250480925,201902,1,0,39,167,2269.95,558.15,685.30,7,...,1.0,0.0,-11730.00,0.0,2205.0,6748.23,12.0,0.0,1630.47,CONTINUA
2,250480925,201903,1,0,39,168,3928.99,897.46,1584.45,7,...,4.0,10557.0,-25731.12,0.0,2236.0,13258.06,18.0,0.0,0.00,CONTINUA
3,250480925,201904,1,0,39,169,3431.69,247.72,1537.30,7,...,-1.0,0.0,-19107.25,0.0,2266.0,17332.37,16.0,0.0,1243.38,CONTINUA
4,250480925,201905,1,0,39,170,0.00,0.00,0.00,7,...,2.0,0.0,-14662.50,0.0,2297.0,8311.09,16.0,0.0,2557.14,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4901232,1590099528,202105,1,0,55,1,134.97,2.63,112.44,7,...,5.0,0.0,NaN,NaN,28.0,NaN,NaN,NaN,0.00,CONTINUA
4901233,1590099528,202106,1,0,55,2,228.19,3.45,19.10,7,...,0.0,0.0,NaN,NaN,58.0,NaN,NaN,NaN,0.00,CONTINUA
4901234,1590099528,202107,1,0,55,3,318.34,1.23,98.19,7,...,3.0,0.0,NaN,NaN,89.0,NaN,NaN,NaN,0.00,CONTINUA
4901235,1590099528,202108,1,0,55,4,469.97,58.16,154.16,7,...,6.0,0.0,NaN,NaN,120.0,NaN,NaN,NaN,0.00,None


In [ ]:
tarjetas_variables = [
    
    ('ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'),
    ('mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'),
    ('ctarjeta_visa_descuentos', 'ctarjeta_master_descuentos'),
    ('mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos'),
   
    ('Visa_msaldototal', 'Master_msaldototal'),
    ('Visa_msaldopesos', 'Master_msaldopesos'),
    ('Visa_msaldodolares', 'Master_msaldodolares'),
    ('Visa_mconsumospesos', 'Master_mconsumospesos'),
    ('Visa_mconsumosdolares', 'Master_mconsumosdolares'),
    ('Visa_mlimitecompra', 'Master_mlimitecompra'),
    ('Visa_madelantopesos', 'Master_madelantopesos'),
    ('Visa_madelantodolares', 'Master_madelantodolares'),
    ('Visa_fultimo_cierre', 'Master_fultimo_cierre'),
    ('Visa_mpagado', 'Master_mpagado'),
    ('Visa_mpagospesos', 'Master_mpagospesos'),
    ('Visa_mpagosdolares', 'Master_mpagosdolares'),
    ('Visa_fechaalta', 'Master_fechaalta'),
    
    ('Visa_cconsumos', 'Master_cconsumos'),
    ('Visa_cadelantosefectivo', 'Master_cadelantosefectivo'),
    ('Visa_mpagominimo', 'Master_mpagominimo')
]


In [ ]:
paired_columns = []
for visa, master in tarjetas_variables:
    paired_columns.append(f"IFNULL({visa}, 0) + IFNULL ({master},0) AS conjunto_{visa}")

paired_columns_sql = ",\n    ".join(paired_columns)





In [ ]:
query = f"""
CREATE OR REPLACE TABLE competencia_03 AS
SELECT *,
    {paired_columns_sql}
FROM competencia_03
"""

%sql {{query}}

,Success


In [ ]:
%%sql 


create or replace table competencia_03 as
select
    *,
    (case when Visa_status = 0 then True end) as T_Visa_normal,
    (case when Master_status = 0 then True end) as T_Master_normal
from competencia_03;


create or replace table competencia_03 as
        select *,
        CASE 
        WHEN Visa_status = 6 then 1
        WHEN Visa_status = 7 then 2 
        WHEN Visa_status = 8 then 3
        ELSE NULL
        END as Visa_status
        from competencia_03;

,Success


In [ ]:
%%sql

alter table competencia_03 drop column visa_status;
alter table competencia_03 drop column Master_status;

,Success


In [ ]:
lag_columns = []
for variable in lista_feature:
    lag_columns.append(f"lag({variable}, 1) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_1")
    lag_columns.append(f"lag({variable}, 2) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_2")


lag_columns_sql = ",\n    ".join(lag_columns)


query = f"""
CREATE OR REPLACE TABLE competencia_03 AS
SELECT *,
    {lag_columns_sql}
FROM competencia_03
"""

%sql {{query}}



,Success


In [ ]:
delta_columns = []
for variable in lista_feature:
    nombre_variable = variable+'_delta_1'
    nombre_variable_2 = variable+'_delta_2'
    delta_columns.append(f"{variable} - {variable}_lag_1 as {nombre_variable}")
    delta_columns.append(f"{variable}_lag_1 - {variable}_lag_2 as {nombre_variable_2}")

delta_columns_sql = ",\n    ".join(delta_columns)


query = f'''CREATE OR REPLACE TABLE competencia_03 AS
            select *,
            {delta_columns_sql}
            from competencia_03'''
%sql {{query}}
            

,Success


In [ ]:
## Ingrese path para guardar dataset

path_dataset_lags_deltas_y_clase_ternaria = datasets_path_l+r'\competencia_03_lags_deltas_y_clase_ternaria.parquet'
path_dataset_lags_deltas_y_clase_ternaria


'C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_3\\competencia_03_lags_deltas_y_clase_ternaria.parquet'

In [ ]:
%sql COPY competencia_03 TO '{{path_dataset_lags_deltas_y_clase_ternaria}}' (FORMAT 'parquet')


,Success


In [ ]:
df_train = pd.read_parquet(dataset_competencia_03_lags_deltas_y_clase_ternaria_l)


In [ ]:
df_train[df_train['foto_mes']==201908]['clase_ternaria'].value_counts()

In [ ]:
df_train[df_train['foto_mes']==201909]['clase_ternaria'].value_counts()

In [ ]:
df_train['foto_mes'].unique()

In [ ]:

df_counts = df_train.groupby(['foto_mes', 'clase_ternaria'])['clase_ternaria'].count()
df_unstacked = df_counts.unstack(fill_value=0)
df_unstacked['diferencia'] = df_unstacked.get('BAJA+2', 0) - df_unstacked.get('BAJA+1', 0)
df_baja = df_unstacked.reset_index()
df_baja.style.background_gradient(cmap='CMRmap_r', axis=0)

In [ ]:

df_baja[df_baja['BAJA+2']>630]['foto_mes'].unique()

In [ ]:
import matplotlib.pyplot as plt

# Agrupar por 'foto_mes' y sumar las bajas 'BAJA+2'
bajas_por_mes = df_baja.groupby('foto_mes')['BAJA+2'].sum()

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(bajas_por_mes.index, bajas_por_mes.values, marker='o', linestyle='-', label='Cantidad de BAJA+2')
plt.xticks(bajas_por_mes.index, rotation=45)
plt.xlabel('Mes')
plt.ylabel('Cantidad de BAJA+2')
plt.title('Cantidad de BAJA+2 por Mes')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [12]:
lista_columnas = df_train.columns
listas_drop = [col for col in lista_columnas if 'lag' in col]
df_train = df_train.drop(columns=listas_drop)

In [15]:
columnas_num = df_train.select_dtypes(exclude=['bool','object'])
lista_columnas_num = columnas_num.columns

df_train['max'] = df_train[lista_columnas_num].max(axis=1)

In [ ]:
variables = [
    'cmobile_app_trx', 'Master_cconsumos', 'ctarjeta_debito_transacciones',
    'ctarjeta_visa_transacciones', 'ctarjeta_master_transacciones', 'cpayroll_trx',
    'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'cpagodeservicios', 'cforex',
    'cforex_buy', 'mforex_sell', 'cextraccion_autoservicio',
    'ccallcenter_transacciones', 'chomebanking_transacciones'
]
if 'ctrx_quarter' not in df_train.columns:
    raise KeyError("La variable 'ctrx_quarter' no existe en el dataset.")
variables_presentes = [var for var in variables if var in df_train.columns]
if variables_presentes:
    for var in variables_presentes:
        denominator = df_train[var].replace(0, 1e-6) ** 2
        df_train['power_ratio_' + var] = df_train['ctrx_quarter'] / denominator
    variables_faltantes = [var for var in variables if var not in df_train.columns]
    if variables_faltantes:
        print(f"Las siguientes variables no existen en el dataset y no se calcularon: {variables_faltantes}")
else:
    print("Ninguna de las variables especificadas existe en el dataset.")

In [17]:
df_train.shape

(2574489, 336)

## Formateo previo

In [18]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [20]:
data = df_train

In [ ]:
data['tmobile_app'] = pd.to_numeric(data['tmobile_app'], errors='coerce')  
data['tmobile_app'] = data['tmobile_app'].fillna(0).astype('bool')  



data['cmobile_app_trx'] = pd.to_numeric(data['cmobile_app_trx'], errors='coerce')  
data['cmobile_app_trx'] = data['cmobile_app_trx'].fillna(0).astype('bool')  

In [22]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
df_test = data[data['foto_mes'] == mes_test]
df_train = data[data['foto_mes'].isin(mes_train)]

clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train.loc[X_train.index, 'clase_peso']


## Entrenamiento y Semillero-Voting GD

In [ ]:
%%capture
# 7 - Voting promedio de modelos para reducir varianza
numeros_random = np.random.randint(0, 100000, 15)
train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)



df_voting = pd.DataFrame()

for semilla in numeros_random: 
        
    
    df_modelos = pd.DataFrame
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': best_params['num_leaves'],
    'learning_rate': best_params['learning_rate'],
    'min_data_in_leaf': best_params['min_data_in_leaf'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    'seed': semilla,
    'verbose': 0
    }
        
        
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    y_pred = model.predict(X_test)
    
    df_voting[f'pred_{semilla}'] = y_pred
    
df_voting['pred_mean'] = df_voting.mean(axis=1)
    

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

## Predicción

In [ ]:
y_pred = df_voting['pred_mean']

predicciones = y_pred

X_test['Probabilidad'] = predicciones

tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

In [ ]:

tb_entrega['Predicted'] = 0

envios = 9730
tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1

resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)

print("Cantidad de clientes {}".format(envios))
num_subida_kaggle = 18
nombre_archivo = '\entrega_0{}.csv'.format(num_subida_kaggle)
entrega_final = f'{entregas_l}{nombre_archivo}'
resultados.to_csv(entrega_final, index=False)

Cantidad de clientes 9730


In [ ]:
numero_de_cliente = X_test['numero_de_cliente']

In [ ]:

nombres = model.feature_name()
df_resultado = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente,
    'Predicted': y
}, index=X_test.index)

In [ ]:

predicciones = y_pred_lgm

X_test['Probabilidad'] = predicciones

tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

cortes = range(9000,14000,100)

num_subida_kaggle = 65
for envios in cortes:
    
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    
    nombre_archivo = 'entrega_0{}.csv'.format(num_subida_kaggle)
    entrega_final = os.path.join(path, nombre_archivo)
    resultados.to_csv(entrega_final, index=False)
    
    
    cantidad_columnas = df_train.shape[1]
    message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}, punto de corte: {envios}, optimizado con optuna: {study_name}"
    
    num_subida_kaggle += 1
    
    
    
    entrega_final = os.path.join(path, nombre_archivo)   
    competencia = 'dm-ey-f-2024-primera'
    try:
        api.competition_submit(file_name=entrega_final,message=message,competition=competencia)
    except:
        print(f"Numero máximo de envios, último envio ={num_subida_kaggle}")